## Deploy in flask web application

In [29]:
from flask import Flask
from werkzeug.serving import run_simple

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == "__main__":
    run_simple('localhost', 5000, app)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2021 11:24:17] "GET / HTTP/1.1" 200 -


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
import pandas as pd
import numpy as np
import io, os, requests

df = pd.read_csv(r'../../all-data/ml/auto-mpg.csv', na_values=['NA','?'])

df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [31]:
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [32]:
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

In [33]:
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [34]:
x = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = df['mpg'].values

In [35]:
type(y)

numpy.ndarray

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [37]:
x_train.shape

(298, 7)

In [38]:
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)

Epoch 1/1000
10/10 - 0s - loss: 2210.1851 - val_loss: 1339.2434
Epoch 2/1000
10/10 - 0s - loss: 519.4469 - val_loss: 558.7532
Epoch 3/1000
10/10 - 0s - loss: 228.9088 - val_loss: 244.6536
Epoch 4/1000
10/10 - 0s - loss: 143.0400 - val_loss: 127.9132
Epoch 5/1000
10/10 - 0s - loss: 104.4002 - val_loss: 92.8554
Epoch 6/1000
10/10 - 0s - loss: 84.3248 - val_loss: 71.1444
Epoch 7/1000
10/10 - 0s - loss: 77.7647 - val_loss: 68.3640
Epoch 8/1000
10/10 - 0s - loss: 79.2557 - val_loss: 71.0825
Epoch 9/1000
10/10 - 0s - loss: 74.3187 - val_loss: 65.3218
Epoch 10/1000
10/10 - 0s - loss: 71.0231 - val_loss: 64.1426
Epoch 11/1000
10/10 - 0s - loss: 69.9714 - val_loss: 63.6566
Epoch 12/1000
10/10 - 0s - loss: 68.2729 - val_loss: 60.6379
Epoch 13/1000
10/10 - 0s - loss: 66.5891 - val_loss: 59.9225
Epoch 14/1000
10/10 - 0s - loss: 65.1534 - val_loss: 59.7299
Epoch 15/1000
10/10 - 0s - loss: 64.4667 - val_loss: 56.7507
Epoch 16/1000
10/10 - 0s - loss: 62.4664 - val_loss: 57.3791
Epoch 17/1000
10/10 - 

In [39]:
prediction = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(prediction, y_test))
print(f'After the training the score is: {score}')

After the training the score is: 5.6406302096197685


In [40]:
os.chdir(r'../../all-data/ml')

In [41]:
os.getcwd()

'/Users/johnmoses/Workextra/essentials/all-data/ml'

In [42]:
model.save(os.path.join(os.getcwd(), "mpg_model.h5"))

In [43]:
cols = [x for x in df.columns if x not in ('mpg', 'name')]
cols
print("{")
for i, name in enumerate(cols):
    print(f'"{name}":{{"min": {df[name].min()}, "max": {df[name].max()}}} {"," if i<(len(cols)-1) else ""}')
print("}")

{
"cylinders":{"min": 3, "max": 8} ,
"displacement":{"min": 68.0, "max": 455.0} ,
"horsepower":{"min": 46.0, "max": 230.0} ,
"weight":{"min": 1613, "max": 5140} ,
"acceleration":{"min": 8.0, "max": 24.8} ,
"year":{"min": 70, "max": 82} ,
"origin":{"min": 1, "max": 3} 
}
